## MongoDB Query Generator using OpenAI JSON mode

### Install the required libraries

In [ ]:
!pip install openai
!pip install python-dotenv
!pip install langchain

### Import the necessary packages

In [1]:
import openai
import os
import json
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

### Load the OpenAI API key as environment variable

The OpenAI API key is present in a .env file with the key **"OPENAI_API_KEY"**

In [2]:
_ = load_dotenv()

### Instantiate the model

bind() function returns a Runnable object. So, we need to invoke the LLM using chat.invoke(). Using the returned object as a function - chat() - wouldn't work on a Runnable. Please refer to the [API documentation](https://api.python.langchain.com/en/stable/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html#langchain_community.chat_models.openai.ChatOpenAI)

Here we are using bind() to pass in the reponse_format supported by latest OpenAI models released as of 11/6/2023, as a kwarg since there is no explicit parameter to the class supported yet.

In [3]:
GPT_MODEL = "gpt-3.5-turbo-1106"
#GPT_MODEL = "gpt-4-1106-preview"

chat = ChatOpenAI(temperature=0, model=GPT_MODEL).bind(
    response_format={"type": "json_object"}
)

### Prompt Template

In [4]:
prompt_template_str = """
Your task is to write a MongoDB Query, specifically an aggregation pipeline \
that would produce the expected output for the given input.

Return a JSON response with the following keys:
```
mongoDBQuery: The MongoDB aggregation pipeline to produce the expected output for a given input.\
This field corresponds to just the list of stages in the aggregation pipeline \
and shouldn't contain the "db.collection.aggregate" prefix.
    
queryExplanation: A detailed explanation for the query that was returned.
```
    
Input data: {input_data} 
Expected output data: {expected_output}
"""

prompt_template = ChatPromptTemplate.from_template(template=prompt_template_str)

In [5]:
def get_llm_response(input_data, expected_output):
    messages = prompt_template.format_messages(input_data = input_data, expected_output = expected_output)
    response = chat.invoke(messages)
    #print(f"response from assistant: {response}")
    return response.content

### Example 1: Simple Group By query
The task here is to write a MongoDB aggregation pipeline to group the data based on team and count the players for each team. 

In [6]:
ex1_input_data = """
[
  {
    "name": "Sachin",
    "team": "India"
  },
  {
    "name": "Sourav",
    "team": "India"
  },
  {
    "name": "Lara",
    "team": "West Indies"
  }
]
"""

ex1_expected_output = """
[
 {
   "team": India,
   "playerCount": 2
 },
 {
   "team": "West Indies",
   "playerCount": 1
 }
]
"""

In [7]:
llm_response = get_llm_response(ex1_input_data, ex1_expected_output)

llm_response_json = json.loads(llm_response)
#print(f"llm_response:\n{llm_response_json}")
print(f"MongoDB Query:\n{llm_response_json.get('mongoDBQuery')}")
print(f"\nQuery Explanation: {llm_response_json.get('queryExplanation')}")

MongoDB Query:
[{'$group': {'_id': '$team', 'playerCount': {'$sum': 1}}}, {'$project': {'_id': 0, 'team': '$_id', 'playerCount': 1}}]

Query Explanation: In this MongoDB aggregation pipeline, we first use the $group stage to group the documents by the 'team' field and calculate the count of players for each team using the $sum accumulator. Then, we use the $project stage to reshape the output and exclude the _id field, while renaming the _id field to 'team' and keeping the 'playerCount' field.


### Example 2: Find the duplicates
The task is to write a MongoDB aggregation pipeline to find the documents that have duplicates in the nested array "courses" and count the number of times those duplicate items are present in the array. 

In [8]:
ex2_input_data = """
[
 {
   "student": "Sachin",
   "courses": [
      {
        "courseName": "batting",
        "marks": 100
      },
      {
        "courseName": "batting",
        "marks": 50
      },
      {
        "courseName": "fielding",
        "marks": 60
      }
   ]
 },
 {
   "student": "Sourav",
   "courses": [
      {
        "courseName": "batting",
        "marks": 80
      },
      {
        "courseName": "bowling",
        "marks": 60
      },
      {
        "courseName": "fielding",
        "marks": 40
      }
   ]
 }
]
"""

ex2_expected_output = """
[
  {
    "student": "Sachin"
    "duplicateCourses": [
      {
        "courseName": "batting"
        "duplicateCount": 2
      }
    ]
  }
]
"""

In [9]:
llm_response = get_llm_response(ex2_input_data, ex2_expected_output)

llm_response_json = json.loads(llm_response)
print(f"llm_response:\n{llm_response_json}")
#print(f"MongoDB Query:\n{llm_response_json.get('mongoDBQuery')}")
#print(f"\nQuery Explanation: {llm_response_json.get('queryExplanation')}")

llm_response:
{'mongoDBQuery': [{'$unwind': '$courses'}, {'$group': {'_id': {'student': '$student', 'courseName': '$courses.courseName'}, 'count': {'$sum': 1}}}, {'$match': {'count': {'$gt': 1}}}, {'$group': {'_id': '$_id.student', 'duplicateCourses': {'$push': {'courseName': '$_id.courseName', 'duplicateCount': '$count'}}}}, {'$project': {'_id': 0, 'student': '$_id', 'duplicateCourses': 1}}], 'queryExplanation': "The aggregation pipeline starts by unwinding the 'courses' array to deconstruct the array into separate documents. Then, it groups the documents by 'student' and 'courseName', and counts the occurrences of each course for each student. After that, it filters out the courses with a count greater than 1, indicating duplicate courses. Next, it regroups the data by 'student' and pushes the duplicate courses into an array. Finally, it projects the output to include only the 'student' and 'duplicateCourses' fields."}


### Example 3 - Group By an attribute which is in the nested array
The task is to group by author name to get the list of books written by them

In [10]:
ex3_input_data = """
[
  {
    "bookCategory": "Non-Fiction",
    "books": [
      {
        "bookName": "Seven Habits",
        "pages": 200,
        "authors": [
          {
            "authorName": "Sachin",
            "authorEmail": "sachin@gmail.com"
          },
          {
            "authorName": "Sourav",
            "authorEmail": "sourav@gmail.com"
          }
        ]
      },
      {
        "bookName": "One thing",
        "pages": 100,
        "authors": [
          {
            "authorName": "Sachin",
            "authorEmail": "sachin@gmail.com"
          }
        ]
      },
      
    ]
  },
  {
    "bookCategory": "Fiction",
    "books": [
      {
        "bookName": "Harry Potter",
        "pages": 400,
        "authors": [
          {
            "authorName": "Sachin",
            "authorEmail": "sachin@gmail.com"
          },
          {
            "authorName": "Tim",
            "authorEmail": "Tim@gmail.com"
          }
        ]
      },
      {
        "bookName": "Alchemist",
        "pages": 100,
        "authors": [
          {
            "authorName": "Sourav",
            "authorEmail": "sourav@gmail.com"
          }
        ]
      },
      
    ]
  }
]
"""

ex3_expected_output = """
[
  {
     "authorName": "Sachin",
     "bookName": [
        "Seven Habits",
        "One thing",
        "Harry Potter"
     ] 
  },
  {
     "authorName": "Sourav",
     "bookName": [
        "Seven Habits",
        "Alchemist"
     ] 
  },
  {
     "authorName": "Tim",
     "bookName": [
        "Harry Potter"
     ] 
  }
]
"""

In [11]:
llm_response = get_llm_response(ex3_input_data, ex3_expected_output)

llm_response_json = json.loads(llm_response)
#print(f"llm_response:\n{llm_response_json}")
print(f"MongoDB Query:\n{llm_response_json.get('mongoDBQuery')}")
print(f"\nQuery Explanation: {llm_response_json.get('queryExplanation')}")

MongoDB Query:
[{'$unwind': '$books'}, {'$unwind': '$books.authors'}, {'$group': {'_id': '$books.authors.authorName', 'bookName': {'$addToSet': '$books.bookName'}}}, {'$project': {'_id': 0, 'authorName': '$_id', 'bookName': 1}}]

Query Explanation: The aggregation pipeline starts by unwinding the 'books' array to deconstruct the array into separate documents. Then, it unwinds the 'authors' array within each book document. Next, it groups the documents by the author's name and uses the $addToSet operator to create an array of unique book names for each author. Finally, it projects the output to rename the '_id' field to 'authorName' and includes the 'bookName' array in the result.


### Example 4: Extract nested array content into one single array

In [12]:
ex4_input_data = """
[
  {
    "bookCategory": "Non-Fiction",
    "books": [
      {
        "bookName": "Seven Habits",
        "pages": 200,
        "authors": [
          {
            "authorName": "Sachin",
            "authorEmail": "sachin@gmail.com"
          },
          {
            "authorName": "Sourav",
            "authorEmail": "sourav@gmail.com"
          }
        ]
      },
      {
        "bookName": "One thing",
        "pages": 100,
        "authors": [
          {
            "authorName": "Sachin",
            "authorEmail": "sachin@gmail.com"
          }
        ]
      },
      
    ]
  },
  {
    "bookCategory": "Fiction",
    "books": [
      {
        "bookName": "Harry Potter",
        "pages": 400,
        "authors": [
          {
            "authorName": "Sachin",
            "authorEmail": "sachin@gmail.com"
          },
          {
            "authorName": "Tim",
            "authorEmail": "Tim@gmail.com"
          }
        ]
      },
      {
        "bookName": "Alchemist",
        "pages": 100,
        "authors": [
          {
            "authorName": "Sourav",
            "authorEmail": "sourav@gmail.com"
          }
        ]
      },
      
    ]
  }
]
"""

ex4_expected_output = """
[
  {
    "authors": [
      {
        "authorEmail": "sachin@gmail.com",
        "authorName": "Sachin"
      },
      {
        "authorEmail": "sourav@gmail.com",
        "authorName": "Sourav"
      }
    ],
    "bookName": "Seven Habits",
    "pages": 200
  },
  {
    "authors": [
      {
        "authorEmail": "sachin@gmail.com",
        "authorName": "Sachin"
      }
    ],
    "bookName": "One thing",
    "pages": 100
  },
  {
    "authors": [
      {
        "authorEmail": "sachin@gmail.com",
        "authorName": "Sachin"
      },
      {
        "authorEmail": "Tim@gmail.com",
        "authorName": "Tim"
      }
    ],
    "bookName": "Harry Potter",
    "pages": 400
  },
  {
    "authors": [
      {
        "authorEmail": "sourav@gmail.com",
        "authorName": "Sourav"
      }
    ],
    "bookName": "Alchemist",
    "pages": 100
  }
]
"""

In [13]:
llm_response = get_llm_response(ex4_input_data, ex4_expected_output)

llm_response_json = json.loads(llm_response)
#print(f"llm_response:\n{llm_response_json}")
print(f"MongoDB Query:\n{llm_response_json.get('mongoDBQuery')}")
print(f"\nQuery Explanation: {llm_response_json.get('queryExplanation')}")

MongoDB Query:
[{'$unwind': '$books'}, {'$project': {'bookName': '$books.bookName', 'pages': '$books.pages', 'authors': '$books.authors'}}, {'$unwind': '$authors'}, {'$group': {'_id': {'bookName': '$bookName', 'pages': '$pages'}, 'authors': {'$addToSet': '$authors'}}}, {'$project': {'_id': 0, 'bookName': '$_id.bookName', 'pages': '$_id.pages', 'authors': 1}}]

Query Explanation: The aggregation pipeline starts by unwinding the 'books' array to deconstruct the array into separate documents. Then, it projects the 'bookName', 'pages', and 'authors' fields. After that, it unwinds the 'authors' array to deconstruct the array into separate documents. Next, it groups the documents by 'bookName' and 'pages', and uses the $addToSet operator to create an array of unique authors for each book. Finally, it projects the final output with 'bookName', 'pages', and 'authors' fields while excluding the _id field.


### Example 5: extract nested array as objects

In [14]:
ex5_input_data = """
[
  {
    "bookName": "Seven Habits",
    "pages": 200,
    "authors": [
      {
        "authorName": "Sachin",
        "authorEmail": "sachin@gmail.com"
      },
      {
        "authorName": "Sourav",
        "authorEmail": "sourav@gmail.com"
      }
    ]
  },
  {
    "bookName": "One thing",
    "pages": 100,
    "authors": [
      {
        "authorName": "Sachin",
        "authorEmail": "sachin@gmail.com"
      }
    ]
  },
  {
    "bookName": "Harry Potter",
    "pages": 400,
    "authors": [
      {
        "authorName": "Sachin",
        "authorEmail": "sachin@gmail.com"
      },
      {
        "authorName": "Tim",
        "authorEmail": "Tim@gmail.com"
      }
    ]
  },
  {
    "bookName": "Alchemist",
    "pages": 100,
    "authors": [
      {
        "authorName": "Sourav",
        "authorEmail": "sourav@gmail.com"
      }
    ]
  }
]
"""

ex5_expected_output = """
[
    {
        "bookName": "Seven Habits",
        "pages": 200,
        "authorName": "Sachin",
        "authorEmail": "sachin@gmail.com"
    },
    {
        "bookName": "Seven Habits",
        "pages": 200,
        "authorName": "Sourav",
        "authorEmail": "sourav@gmail.com"
    },
    {
        "bookName": "One thing",
        "pages": 100,
        "authorName": "Sachin",
        "authorEmail": "sachin@gmail.com"
    },
    {
        "bookName": "Harry Potter",
        "pages": 400,
        "authorName": "Sachin",
        "authorEmail": "sachin@gmail.com"
    },
    {
        "bookName": "Harry Potter",
        "pages": 400,
        "authorName": "Tim",
        "authorEmail": "Tim@gmail.com"
    },
    {
        "bookName": "Alchemist",
        "pages": 100,
        "authorName": "Sourav",
        "authorEmail": "sourav@gmail.com"
    }
]
"""

In [15]:
llm_response = get_llm_response(ex5_input_data, ex5_expected_output)

llm_response_json = json.loads(llm_response)
#print(f"llm_response:\n{llm_response_json}")
print(f"MongoDB Query:\n{llm_response_json.get('mongoDBQuery')}")
print(f"\nQuery Explanation: {llm_response_json.get('queryExplanation')}")

MongoDB Query:
[{'$unwind': '$authors'}, {'$project': {'bookName': 1, 'pages': 1, 'authorName': '$authors.authorName', 'authorEmail': '$authors.authorEmail', '_id': 0}}]

Query Explanation: The aggregation pipeline starts by using the $unwind stage to deconstruct the authors array, creating a separate document for each element in the array. Then, the $project stage is used to reshape the documents, including only the required fields such as bookName, pages, authorName, and authorEmail. The _id field is excluded from the output using '_id': 0.


### Example 6 - Aggregate and produce a summary

In [16]:
ex6_input_data = """
[
    {
        "studentName": "Pete",
        "subjects": [
            {
                "subjectName": "Math",
                "result": "passed"
            },
            {
                "subjectName": "Physics",
                "result": "passed"
            },
            {
                "subjectName": "Chemistry",
                "result": "failed"
            },
            {
                "subjectName": "Botany",
                "result": "failed"
            },
            {
                "subjectName": "Zoology",
                "result": "failed"
            }
        ]
    }
]
"""

ex6_expected_output = """
{
    "totalSubjectCount": 5,
    "totalResultPassedCount": 2,
    "totalResultFailedCount": 3
}
"""

In [17]:
llm_response = get_llm_response(ex6_input_data, ex6_expected_output)

llm_response_json = json.loads(llm_response)
#print(f"llm_response:\n{llm_response_json}")
print(f"MongoDB Query:\n{llm_response_json.get('mongoDBQuery')}")
print(f"\nQuery Explanation: {llm_response_json.get('queryExplanation')}")

MongoDB Query:
[{'$unwind': '$subjects'}, {'$group': {'_id': None, 'totalSubjectCount': {'$sum': 1}, 'totalResultPassedCount': {'$sum': {'$cond': [{'$eq': ['$subjects.result', 'passed']}, 1, 0]}}, 'totalResultFailedCount': {'$sum': {'$cond': [{'$eq': ['$subjects.result', 'failed']}, 1, 0]}}}}, {'$project': {'_id': 0, 'totalSubjectCount': 1, 'totalResultPassedCount': 1, 'totalResultFailedCount': 1}}]

Query Explanation: The aggregation pipeline starts by unwinding the 'subjects' array to create a separate document for each subject. Then, it groups the documents to calculate the total count of subjects, the count of passed results, and the count of failed results using the $sum operator and $cond to conditionally count the results. Finally, the $project stage is used to reshape the output and remove the _id field.


### Import the packages necessary for Langchain Output parser

In [28]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
import re

In [25]:
chat_openai = ChatOpenAI(temperature=0, model=GPT_MODEL)

In [33]:
prompt_template_with_format_instructions_str = """
Your task is to write a MongoDB Query, specifically an aggregation pipeline \
that would produce the expected output for the given input.

mongodb_query: The MongoDB aggregation pipeline to produce the expected output for a given input.\
This field corresponds to just the list of stages in the aggregation pipeline \
and shouldn't contain the "db.collection.aggregate" prefix.

query_explanation: Explanation of what the query is doing.

Format the output as JSON with the following keys:
mongodb_query
query_explanation

Input data: {input_data}
Expected output: {expected_output}

{format_instructions}
"""

prompt_template_with_format_instructions = ChatPromptTemplate.from_template(template=prompt_template_with_format_instructions_str)

### Response Schema & Output Instructions

In [37]:
mongodb_query_schema = \
    ResponseSchema(name="mongodb_query",
                   description="The MongoDB aggregation pipeline to produce the expected output for a given input.\
This field corresponds to just the list of stages in the aggregation pipeline and shouldn't contain the 'db.collection.aggregate' prefix.")

query_explanation_schema = ResponseSchema(name="query_explanation",
                                          description="Explanation of what the query is doing.")

response_schemas = [mongodb_query_schema, query_explanation_schema]

In [38]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"mongodb_query": string  // The MongoDB aggregation pipeline to produce the expected output for a given input.This field corresponds to just the list of stages in the aggregation pipeline and shouldn't contain the 'db.collection.aggregate' prefix.
	"query_explanation": string  // Explanation of what the query is doing.
}
```


### Define a generic function to chat with AI assistant for various input

In [39]:
def get_llm_response(input_data, expected_output, format_instructions):
    messages = prompt_template_with_format_instructions.format_messages(input_data = input_data,
                                                                        expected_output = expected_output,
                                                                        format_instructions = format_instructions)
    response = chat_openai(messages)
    print(f"LLM Response:\n{response.content}")

    json_str = pre_process_json_str(response.content)
    #print(f"Pre-processed JSON str:\n{json_str}")
    
    output_dict = output_parser.parse(json_str)
    return output_dict

def pre_process_json_str(json_str):
    json_str = re.sub(r"\n", '', json_str)
    json_str = re.sub(r"\r", '', json_str)
    json_str = re.sub(r"\t", '', json_str)
    return json_str

In [40]:
llm_response = get_llm_response(ex1_input_data, ex1_expected_output, format_instructions)

print(f"MongoDB Query:\n{llm_response.get('mongodb_query')}")
print(f"\nQuery Explanation: {llm_response.get('query_explanation')}")

LLM Response:
```json
{
	"mongodb_query": [
		{ 
			"$group": {
				"_id": "$team",
				"playerCount": { "$sum": 1 }
			}
		},
		{
			"$project": {
				"_id": 0,
				"team": "$_id",
				"playerCount": 1
			}
		}
	],
	"query_explanation": "The aggregation pipeline first groups the documents by the 'team' field and calculates the count of players for each team using the $group stage. Then, the $project stage is used to reshape the output and exclude the _id field."
}
```
MongoDB Query:
[{'$group': {'_id': '$team', 'playerCount': {'$sum': 1}}}, {'$project': {'_id': 0, 'team': '$_id', 'playerCount': 1}}]

Query Explanation: The aggregation pipeline first groups the documents by the 'team' field and calculates the count of players for each team using the $group stage. Then, the $project stage is used to reshape the output and exclude the _id field.


### Unit test for pre_process_json_str function

In [ ]:
import re
value = """{
	"mongodb_query": "db.collection.aggregate([
		{ $unwind: '$books' },
		{ $project: {
			bookName: '$books.bookName',
			pages: '$books.pages',
			authors: '$books.authors'
		} }
	])",
	"query_explanation": "The query starts by using the $unwind operator to flatten the 'books' array. Then, the $project operator is used to select the desired fields 'bookName', 'pages', and 'authors' from the flattened documents. The result is the expected output."
}"""
value = re.sub(r"\n", '', value)
value = re.sub(r"\r", '', value)
value = re.sub(r"\t", '', value)
value = re.sub(r'(?<!\\)"', r"\"", value)
print(value)